In [ ]:
!pip install csvkit

In [ ]:
!pip install ipython-sql


     ---------------------------------------- 2.0/2.0 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 41.2/41.2 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.39
    Uninstalling SQLAlchemy-1.4.39:
      Successfully uninstalled SQLAlchemy-1.4.39


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
agate-sql 0.5.9 requires sqlalchemy<2, but you have sqlalchemy 2.0.19 which is incompatible.


In [ ]:
import sqlite3

conn = sqlite3.connect('debt_stats.db')
conn.close()


In [ ]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)


C:\Users\MSI\Documents\International_Debt


In [ ]:
%sql sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db


In [ ]:
%%sql

-- Create the table structure
CREATE TABLE international_debt (
    country_name TEXT,
    country_code TEXT,
    indicator_name TEXT,
    indicator_code TEXT,
    debt NUMERIC
);

-- Import data from CSV
.mode csv
.import international_debt.csv international_debt


 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
(sqlite3.OperationalError) table international_debt already exists
[SQL: -- Create the table structure
CREATE TABLE international_debt (
    country_name TEXT,
    country_code TEXT,
    indicator_name TEXT,
    indicator_code TEXT,
    debt NUMERIC
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## 1. <b>Analyzing International Debt Data from The World Bank </b>
<p>When it comes to managing economies, countries, just like individuals, might also resort to taking on debts. One significant avenue for countries to secure financing is through organizations like  <a href="https://www.worldbank.org">The World Bank</a>, which provides loans to nations for various purposes, such as infrastructure development. In this notebook, we'll delve into the analysis of international debt data that has been compiled by The World Bank.</p>

<h4>Objective:</h4>
<p>In this analysis, we aim to explore and extract insights from a dataset containing information about the debt amounts (in USD) owed by developing countries across different debt categories. By querying this dataset, we will seek answers to questions like:</p>
<ul>
<li>What is the total accumulated debt across all countries listed in the dataset?</li>
<li>Which specific country has the highest debt, and what is the corresponding debt amount?</li>
<li>How does the average debt amount vary across different debt indicators?</li>
</ul>
<p><img src="https://assets.datacamp.com/production/project_754/img/image.jpg" alt></p>
<p>The first line of code connects us to the <code>international_debt</code> database where the table <code>international_debt</code> is residing. Let's first <code>SELECT</code> <em>all</em> of the columns from the <code>international_debt</code> table. Also, we'll limit the output to the first ten rows to keep the output clean.</p>


In [ ]:
%%sql

-- Select and display the first 10 rows
SELECT * FROM international_debt LIMIT 10;


 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


country_name,country_code,indicator_name,indicator_code,debt
Afghanistan,AFG,"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD,72894453.7
Afghanistan,AFG,"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD,53239440.1
Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9
Afghanistan,AFG,"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD,49114729.4
Afghanistan,AFG,"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD,39903620.1
Afghanistan,AFG,"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD,39107845.0
Afghanistan,AFG,"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD,23779724.3
Afghanistan,AFG,"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD,13335820.0
Afghanistan,AFG,"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD,100847181.9
Afghanistan,AFG,"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD,72894453.7


## 2. Finding the number of distinct countries
<p>From the first ten rows, we can see the amount of debt owed by <em>Afghanistan</em> in the different debt indicators. But we do not know the number of different countries we have on the table. There are repetitions in the country names because a country is most likely to have debt in more than one debt indicator. </p>
<p>Without a count of unique countries, we will not be able to perform our statistical analyses holistically. In this section, we are going to extract the number of unique countries present in the table. </p>

In [ ]:
%%sql
SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries
FROM international_debt;


 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


total_distinct_countries
124


## 3. Finding out the distinct debt indicators
<p>We can see there are a total of 124 countries present on the table. As we saw in the first section, there is a column called <code>indicator_name</code> that briefly specifies the purpose of taking the debt. Just beside that column, there is another column called <code>indicator_code</code> which symbolizes the category of these debts. Knowing about these various debt indicators will help us to understand the areas in which a country can possibly be indebted to. </p>

In [ ]:
%%sql
SELECT DISTINCT(indicator_code) AS distinct_debt_indicators
From international_debt
Order By 1;

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


distinct_debt_indicators
DT.AMT.BLAT.CD
DT.AMT.DLXF.CD
DT.AMT.DPNG.CD
DT.AMT.MLAT.CD
DT.AMT.OFFT.CD
DT.AMT.PBND.CD
DT.AMT.PCBK.CD
DT.AMT.PROP.CD
DT.AMT.PRVT.CD
DT.DIS.BLAT.CD


## 4. Totaling the amount of debt owed by the countries
<p>As mentioned earlier, the financial debt of a particular country represents its economic state. But if we were to project this on an overall global scale, how will we approach it?</p>
<p>Let's switch gears from the debt indicators now and find out the total amount of debt (in USD) that is owed by the different countries. This will give us a sense of how the overall economy of the entire world is holding up.</p>

In [ ]:
%%sql
SELECT
Round(sum(debt)/1000000,2) as total_debt
from international_debt;

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


total_debt
3079734.49


## 5. Country with the highest debt
<p>"Human beings cannot comprehend very large or very small numbers. It would be useful for us to acknowledge that fact." - <a href="https://en.wikipedia.org/wiki/Daniel_Kahneman">Daniel Kahneman</a>. That is more than <em>3 million <strong>million</strong></em> USD, an amount which is really hard for us to fathom. </p>
<p>Now that we have the exact total of the amounts of debt owed by several countries, let's now find out the country that owns the highest amount of debt along with the amount. <strong>Note</strong> that this debt is the sum of different debts owed by a country across several categories. This will help to understand more about the country in terms of its socio-economic scenarios. We can also find out the category in which the country owns its highest debt. But we will leave that for now. </p>

In [ ]:
%%sql

select
country_name, sum(debt) as total_debt
from international_debt
group by country_name
order by total_debt DESC
limit 1;

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


country_name,total_debt
China,285793494734.2


## 6. Average amount of debt across indicators
<p>So, it was <em>China</em>. A more in-depth breakdown of China's debts can be found <a href="https://datatopics.worldbank.org/debt/ids/country/CHN">here</a>. </p>
<p>We now have a brief overview of the dataset and a few of its summary statistics. We already have an idea of the different debt indicators in which the countries owe their debts. We can dig even further to find out on an average how much debt a country owes? This will give us a better sense of the distribution of the amount of debt across different indicators.</p>

In [ ]:
%%sql
select
    indicator_code as debt_indicator,
    indicator_name,
    avg(debt) as average_debt
from international_debt
group by debt_indicator, indicator_name
order by average_debt DESC
limit 10;

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


debt_indicator,indicator_name,average_debt
DT.AMT.DLXF.CD,"Principal repayments on external debt, long-term (AMT, current US$)",5904868401.499195
DT.AMT.DPNG.CD,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",5161194333.812659
DT.DIS.DLXF.CD,"Disbursements on external debt, long-term (DIS, current US$)",2152041216.8902435
DT.DIS.OFFT.CD,"PPG, official creditors (DIS, current US$)",1958983452.8598351
DT.AMT.PRVT.CD,"PPG, private creditors (AMT, current US$)",1803694101.963265
DT.INT.DLXF.CD,"Interest payments on external debt, long-term (INT, current US$)",1644024067.6508067
DT.DIS.BLAT.CD,"PPG, bilateral (DIS, current US$)",1223139290.39823
DT.INT.DPNG.CD,"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",1220410844.4215188
DT.AMT.OFFT.CD,"PPG, official creditors (AMT, current US$)",1191187963.083064
DT.AMT.PBND.CD,"PPG, bonds (AMT, current US$)",1082623947.6536236


## 7. The highest amount of principal repayments
<p>We can see that the indicator <code>DT.AMT.DLXF.CD</code> tops the chart of average debt. This category includes repayment of long term debts. Countries take on long-term debt to acquire immediate capital. More information about this category can be found <a href="https://datacatalog.worldbank.org/principal-repayments-external-debt-long-term-amt-current-us-0">here</a>. </p>
<p>An interesting observation in the above finding is that there is a huge difference in the amounts of the indicators after the second one. This indicates that the first two indicators might be the most severe categories in which the countries owe their debts.</p>
<p>We can investigate this a bit more so as to find out which country owes the highest amount of debt in the category of long term debts (<code>DT.AMT.DLXF.CD</code>). Since not all the countries suffer from the same kind of economic disturbances, this finding will allow us to understand that particular country's economic condition a bit more specifically. </p>

In [ ]:
%%sql

select
    country_name,
    indicator_name
from international_debt
where debt = (Select
                     max(debt)
                from international_debt
                where indicator_code like 'DT.AMT.DLXF.CD');

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


country_name,indicator_name
China,"Principal repayments on external debt, long-term (AMT, current US$)"


## 8. The most common debt indicator
<p>China has the highest amount of debt in the long-term debt (<code>DT.AMT.DLXF.CD</code>) category. This is verified by <a href="https://data.worldbank.org/indicator/DT.AMT.DLXF.CD?end=2018&most_recent_value_desc=true">The World Bank</a>. It is often a good idea to verify our analyses like this since it validates that our investigations are correct. </p>
<p>We saw that long-term debt is the topmost category when it comes to the average amount of debt. But is it the most common indicator in which the countries owe their debt? Let's find that out. </p>

In [ ]:
%%sql
select indicator_code, count(indicator_code) as indicator_count
from international_debt
group by indicator_code
order by indicator_count DESC, indicator_code DESC
limit 20;

 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


indicator_code,indicator_count
DT.INT.OFFT.CD,124
DT.INT.MLAT.CD,124
DT.INT.DLXF.CD,124
DT.AMT.OFFT.CD,124
DT.AMT.MLAT.CD,124
DT.AMT.DLXF.CD,124
DT.DIS.DLXF.CD,123
DT.INT.BLAT.CD,122
DT.DIS.OFFT.CD,122
DT.AMT.BLAT.CD,122


## 9. Other viable debt issues and conclusion
<p>There are a total of six debt indicators in which all the countries listed in our dataset have taken debt. The indicator <code>DT.AMT.DLXF.CD</code> is also there in the list. So, this gives us a clue that all these countries are suffering from a common economic issue. But that is not the end of the story, a part of the story rather. </p>
<p>Let's change tracks from <code>debt_indicator</code>s now and focus on the amount of debt again. Let's find out the maximum amount of debt across the indicators along with the respective country names. With this, we will be in a position to identify the other plausible economic issues a country might be going through. By the end of this section, we will have found out the debt indicators in which a country owes its highest debt. </p>
<p>In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.</p>

In [ ]:
%%sql

SELECT country_name, indicator_name, MAX(debt) AS maximum_debt
FROM international_debt
GROUP BY country_name, indicator_name
order by maximum_debt DESC
limit 20;


 * sqlite:///C:\Users\MSI\Documents\International_Debt/debt_stats.db
Done.


country_name,indicator_name,maximum_debt
China,"Principal repayments on external debt, long-term (AMT, current US$)",96218620835.7
Brazil,"Principal repayments on external debt, long-term (AMT, current US$)",90041840304.1
China,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",72392986213.8
Russian Federation,"Principal repayments on external debt, long-term (AMT, current US$)",66589761833.5
Turkey,"Principal repayments on external debt, long-term (AMT, current US$)",51555031005.8
South Asia,"Principal repayments on external debt, long-term (AMT, current US$)",48756295898.2
Brazil,"PPG, private creditors (AMT, current US$)",43598697498.6
Russian Federation,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",42800154974.9
Brazil,"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",41831444053.3
Least developed countries: UN classification,"Disbursements on external debt, long-term (DIS, current US$)",40160766261.6
